# AlexNet

# Alexnet con Torch

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

In [2]:
# Definimos una secuencia de transformaciones:
# - Redimensiona las imágenes a 227x227.
# - Convierte las imágenes a tensores.
# - Normaliza los valores de los píxeles para que estén en el rango [-1,1].
transform = transforms.Compose([
    transforms.Resize((227, 227)),  # Redimensionar a 227x227
    transforms.ToTensor(),          # Convertir a tensor (valores en [0,1])
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))  # Normalización a [-1,1]
])

In [3]:
# Descargar CIFAR-10 (imágenes de 32x32) y aplicar las transformaciones definidas.
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

100%|██████████| 170M/170M [00:03<00:00, 50.3MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
# Definimos el tamaño del batch.
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [5]:
# Definimos el modelo AlexNet

class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            # Capa 1: Convolución (64 filtros, kernel 11, stride 4, padding 2), ReLU y max pooling.
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            # Capa 2: Convolución (192 filtros, kernel 5, padding 2), ReLU y max pooling.
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            # Capa 3: Convolución (384 filtros, kernel 3, padding 1) y ReLU.
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            # Capa 4: Convolución (256 filtros, kernel 3, padding 1) y ReLU.
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            # Capa 5: Convolución (256 filtros, kernel 3, padding 1), ReLU y max pooling.
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.classifier = nn.Sequential(
            # Capa densa 1: 4096 neuronas, con dropout y ReLU.
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),  # Nota: 6x6 es el tamaño espacial tras las operaciones con entrada 227x227.
            nn.ReLU(inplace=True),

            # Capa densa 2: 4096 neuronas, con dropout y ReLU.
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),

            # Capa de salida: 10 neuronas para 10 clases.
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)          # Propagación a través de las capas convolucionales.
        x = x.view(x.size(0), -1)       # Aplanar para pasar a las capas densas.
        x = self.classifier(x)          # Propagación a través de las capas densas.
        return x


In [6]:
# Se usa GPU si está disponible; de lo contrario, se usa CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_torch = AlexNet(num_classes=10).to(device)

print("Se utilizará : ", device)

Se utilizará :  cuda


In [7]:
# Definir la función de pérdida (CrossEntropyLoss) y el optimizador (SGD con lr=0.01 y momentum=0.9).
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_torch.parameters(), lr=0.01, momentum=0.9)
num_epochs = 10  # Número de épocas de entrenamiento

In [8]:
# Entrenamiento del modelo
start_time = time.time()
print("Entrenando AlexNet en PyTorch...")

for epoch in range(num_epochs):
    model_torch.train()  # Poner el modelo en modo entrenamiento (activa dropout, etc.)
    running_loss = 0.0
    correct = 0
    total = 0

    # Iterar sobre cada batch
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Mover datos al dispositivo
        optimizer.zero_grad()            # Reiniciar los gradientes
        outputs = model_torch(inputs)      # Propagación hacia adelante
        loss = criterion(outputs, labels)  # Calcular la pérdida
        loss.backward()                    # Propagación hacia atrás
        optimizer.step()                   # Actualizar los pesos

        running_loss += loss.item() * inputs.size(0)  # Acumular la pérdida
        _, predicted = outputs.max(1)      # Obtener las predicciones
        total += labels.size(0)            # Contar el total de ejemplos
        correct += predicted.eq(labels).sum().item()  # Contar aciertos

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    print(f'Época {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_acc:.4f}')

end_time = time.time()
print(f"Tiempo total de entrenamiento (PyTorch): {end_time - start_time:.2f} segundos")


Entrenando AlexNet en PyTorch...
Época 1/10 - Loss: 1.9196 - Accuracy: 0.2747
Época 2/10 - Loss: 1.2902 - Accuracy: 0.5316
Época 3/10 - Loss: 0.9772 - Accuracy: 0.6547
Época 4/10 - Loss: 0.7983 - Accuracy: 0.7223
Época 5/10 - Loss: 0.6752 - Accuracy: 0.7650
Época 6/10 - Loss: 0.5761 - Accuracy: 0.7999
Época 7/10 - Loss: 0.4960 - Accuracy: 0.8281
Época 8/10 - Loss: 0.4261 - Accuracy: 0.8529
Época 9/10 - Loss: 0.3716 - Accuracy: 0.8712
Época 10/10 - Loss: 0.3231 - Accuracy: 0.8860
Tiempo total de entrenamiento (PyTorch): 438.89 segundos


In [9]:
# Evaluando el modelo
model_torch.eval()  # Poner el modelo en modo evaluación
correct = 0
total = 0
with torch.no_grad():  # Desactivar el cálculo de gradientes
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_torch(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_accuracy = correct / total
print(f'Precisión en test (PyTorch): {test_accuracy * 100:.2f}%')


Precisión en test (PyTorch): 82.78%


# AlexNet en Keras

In [10]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Activation
import time

In [11]:
# Cargamos CIFAR-10: imágenes de 32x32 y etiquetas.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [12]:
# Función de preprocesamiento idéntica a la que se hizo en Torch:
# - Redimensiona cada imagen a 227x227.
# - Convierte la imagen a float32 y normaliza a [0,1].
# - Normaliza a [-1,1] para que coincida con la normalización de PyTorch.
def preprocess(image, label):
    image = tf.image.resize(image, (227, 227))       # Redimensionar a 227x227
    image = tf.cast(image, tf.float32) / 255.0         # Normalizar a [0,1]
    image = (image - 0.5) / 0.5                        # Escalar a [-1,1]
    return image, label

In [13]:
# Convertir las etiquetas a one-hot encoding (para 10 clases)
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test  = tf.keras.utils.to_categorical(y_test, num_classes)

In [14]:
# Crear el pipeline de datos con tf.data:
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

In [15]:
# Definimos el modelo AlexNet

def create_alexnet(input_shape=(227, 227, 3), num_classes=10):
    model = Sequential()
    # Capa 1: Convolución (64 filtros, kernel 11, stride 4, padding 'same'), ReLU y max pooling.
    model.add(Conv2D(64, kernel_size=11, strides=4, padding='same', input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=3, strides=2))

    # Capa 2: Convolución (192 filtros, kernel 5, padding 'same'), ReLU y max pooling.
    model.add(Conv2D(192, kernel_size=5, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=3, strides=2))

    # Capa 3: Convolución (384 filtros, kernel 3, padding 'same') y ReLU.
    model.add(Conv2D(384, kernel_size=3, padding='same'))
    model.add(Activation('relu'))

    # Capa 4: Convolución (256 filtros, kernel 3, padding 'same') y ReLU.
    model.add(Conv2D(256, kernel_size=3, padding='same'))
    model.add(Activation('relu'))

    # Capa 5: Convolución (256 filtros, kernel 3, padding 'same'), ReLU y max pooling.
    model.add(Conv2D(256, kernel_size=3, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=3, strides=2))

    # Aplanar la salida de las capas convolucionales para conectarla a las densas.
    model.add(Flatten())

    # Capa densa 1: 4096 neuronas, con ReLU y dropout.
    model.add(Dense(4096))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    # Capa densa 2: 4096 neuronas, con ReLU y dropout.
    model.add(Dense(4096))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    # Capa de salida: 10 neuronas con activación softmax (para clasificación en 10 clases).
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    return model


In [16]:
# Crear el modelo AlexNet.
model_keras = create_alexnet(input_shape=(227, 227, 3), num_classes=10)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
# Compilamos el modelo utilizando el optimizador SGD (lr=0.01, momentum=0.9),
# la función de pérdida categorical_crossentropy y la métrica accuracy.
model_keras.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

# Entrenamos el modelo por 10 épocas, utilizando el pipeline tf.data para el entrenamiento y validación.
num_epochs = 10

In [18]:
start_time = time.time()
print("Entrenando AlexNet en Keras...")
history = model_keras.fit(train_dataset,
                          epochs=num_epochs,
                          validation_data=test_dataset)
end_time = time.time()
print(f"Tiempo total de entrenamiento (Keras): {end_time - start_time:.2f} segundos")

Entrenando AlexNet en Keras...
Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 39s 35ms/step - accuracy: 0.3096 - loss: 1.8496 - val_accuracy: 0.5633 - val_loss: 1.2183
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - accuracy: 0.5880 - loss: 1.1546 - val_accuracy: 0.6973 - val_loss: 0.8966
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - accuracy: 0.7021 - loss: 0.8603 - val_accuracy: 0.7253 - val_loss: 0.8066
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - accuracy: 0.7569 - loss: 0.7049 - val_accuracy: 0.7630 - val_loss: 0.7146
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - accuracy: 0.7959 - loss: 0.5922 - val_accuracy: 0.7453 - val_loss: 0.7571
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - accuracy: 0.8272 - loss: 0.4962 - val_accuracy: 0.7946 - val_loss: 0.6229
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - accuracy: 0.8551 - loss: 0.4162 - val_accuracy: 0.7830 - val_loss: 0.6900
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - acc

In [19]:
# Evaluación del Modelo en el Conjunto de Test
test_loss, test_acc = model_keras.evaluate(test_dataset, verbose=0)
print(f'Precisión en test (Keras): {test_acc * 100:.2f}%')

Precisión en test (Keras): 79.13%


# Modelos Pre entrenados

## Torch pre entrenado

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import time

In [21]:
# Se utiliza el preprocesamiento estándar de ImageNet:
# - Resize: Redimensiona el lado corto a 256 píxeles.
# - CenterCrop: Recorta al centro a 227x227 (la versión original de AlexNet usaba 227x227).
# - ToTensor: Convierte la imagen a tensor con valores en [0,1].
# - Normalize: Normaliza la imagen usando la media y desviación estándar de ImageNet.
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(227),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [22]:
# Cargar CIFAR-10. Aunque las imágenes son 32x32, se redimensionarán según el transform.
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [23]:
# Cargar AlexNet preentrenado en ImageNet
model = models.alexnet(pretrained=True)

# Adaptar la última capa (la séptima capa del clasificador) para que tenga 10 salidas en lugar de 1000.
model.classifier[6] = nn.Linear(4096, 10)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:02<00:00, 113MB/s]


In [24]:
# Mover el modelo al dispositivo (GPU si está disponible).
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [25]:
# Definimos la Función de Pérdida y el Optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [26]:
# Fine-tuneamos del Modelo Preentrenado en CIFAR-10

num_epochs = 10
start_time = time.time()
model.train()  # Poner el modelo en modo entrenamiento

print("Iniciando fine-tuning del modelo preentrenado en CIFAR-10...")
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()            # Reiniciar los gradientes
        outputs = model(inputs)          # Forward pass
        loss = criterion(outputs, labels)  # Calcular la pérdida
        loss.backward()                  # Backpropagation
        optimizer.step()                 # Actualizar parámetros

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    print(f'Epoch {epoch+1}/{num_epochs}: Loss = {epoch_loss:.4f}, Accuracy = {epoch_acc:.4f}')

end_time = time.time()
print(f"Tiempo total de entrenamiento: {end_time - start_time:.2f} segundos")


Iniciando fine-tuning del modelo preentrenado en CIFAR-10...
Epoch 1/10: Loss = 0.8871, Accuracy = 0.6911
Epoch 2/10: Loss = 0.5680, Accuracy = 0.8045
Epoch 3/10: Loss = 0.4446, Accuracy = 0.8488
Epoch 4/10: Loss = 0.3769, Accuracy = 0.8704
Epoch 5/10: Loss = 0.3233, Accuracy = 0.8889
Epoch 6/10: Loss = 0.2742, Accuracy = 0.9048
Epoch 7/10: Loss = 0.2387, Accuracy = 0.9183
Epoch 8/10: Loss = 0.2066, Accuracy = 0.9289
Epoch 9/10: Loss = 0.1930, Accuracy = 0.9337
Epoch 10/10: Loss = 0.1772, Accuracy = 0.9396
Tiempo total de entrenamiento: 473.11 segundos


In [27]:
model.eval()  # Modo evaluación para desactivar dropout, etc.
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_acc = correct / total
print(f'Precisión en test: {test_acc*100:.2f}%')

Precisión en test: 86.11%


## Keras pre entrenado

El modelo elegido para keras no es AlexNet, ya que no existe nativamente un modelo preentrenado con esta arquitectura. En su defecto, se optó por una vgg16 la cual es "similar" a AlexNet.

In [28]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Activation, Input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import SGD

In [29]:
# CIFAR-10 tiene imágenes de 32x32; las redimensionamos a 224x224 (tamaño esperado para VGG16)
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convertir las etiquetas a one-hot encoding para 10 clases
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test  = to_categorical(y_test, num_classes)

In [30]:
# Función de preprocesamiento:
# - Redimensiona la imagen a 224x224
# - Convierte la imagen a float32
# - Aplica el preprocesamiento de VGG16 (que asume imágenes con rango [0,255] y resta la media de ImageNet)

def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32)
    image = vgg_preprocess(image)  # Esto normaliza la imagen correctamente
    return image, label


In [31]:
# Crear el pipeline de tf.data
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [32]:
# Modelo VGG16 (fine-tuning)
def build_vgg16_finetune(input_shape=(224,224,3), num_classes=10):
    # Cargamos el modelo base VGG16 preentrenado en ImageNet sin la parte superior (include_top=False)
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    # Congelamos las capas base para empezar (fine-tuning de solo las capas superiores)
    for layer in base_model.layers:
        layer.trainable = False

    # Construimos la parte superior personalizada
    x = base_model.output
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [33]:
# Instanciar el modelo
vgg16_model = build_vgg16_finetune()

# Usamos el mismo optimizador (SGD con lr=0.01 y momentum=0.9) y la misma función de pérdida.
optimizer = SGD(learning_rate=1e-4, momentum=0.9)

vgg16_model.compile(optimizer=optimizer,
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [34]:
num_epochs = 10

# Entrenar y evaluar VGG16 fine-tune
print("\nEntrenando VGG16 (fine-tuning):")
history_vgg16 = vgg16_model.fit(train_dataset,
                                epochs=num_epochs,
                                validation_data=test_dataset)


Entrenando VGG16 (fine-tuning):
Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 51s 50ms/step - accuracy: 0.5342 - loss: 3.8659 - val_accuracy: 0.8325 - val_loss: 0.5731
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 36s 43ms/step - accuracy: 0.7672 - loss: 0.9164 - val_accuracy: 0.8471 - val_loss: 0.4813
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 36s 43ms/step - accuracy: 0.8147 - loss: 0.6299 - val_accuracy: 0.8549 - val_loss: 0.4523
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 36s 43ms/step - accuracy: 0.8464 - loss: 0.4882 - val_accuracy: 0.8616 - val_loss: 0.4306
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 36s 43ms/step - accuracy: 0.8684 - loss: 0.3965 - val_accuracy: 0.8656 - val_loss: 0.4187
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 36s 43ms/step - accuracy: 0.8860 - loss: 0.3346 - val_accuracy: 0.8692 - val_loss: 0.4158
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 37s 43ms/step - accuracy: 0.9018 - loss: 0.2851 - val_accuracy: 0.8733 - val_loss: 0.4076
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 37s 43ms/step - a

In [35]:
vgg16_loss, vgg16_acc = vgg16_model.evaluate(test_dataset)
print("Precisión en test de VGG16: {:.2f}%".format(vgg16_acc * 100))

157/157 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.8754 - loss: 0.4113
Precisión en test de VGG16: 87.96%


In [36]:
vgg16_model.summary()

Model: "functional_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 4096)                │     102,764,544 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 253,888,342 (968.51 MB)

 Trainable params: 119,586,826 (456.19 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 119,586,828 (456.19 MB)